In [42]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import scipy.sparse as sp
import os
import json

In [2]:
os.chdir('/Users/jf3375/PycharmProjects/Vatic_Local/Vatic/vatic_test/output/gurobi_output')

## Use Numpy Array to Build Gurobi Models
https://www.gurobi.com/documentation/9.5/examples/matrix1_py.html

In [50]:
#create and output log file: need to put very beginning to record all inputs
m.setParam('LogFile', 'matrix.log')

# Create a new model
m = gp.Model("matrix1")


# Create variables
x = m.addMVar(shape=3, name="x")

# Set objective
obj = np.array([1.0, 1.0, 2.0])
m.setObjective(obj @ x, GRB.MAXIMIZE)

# Build (sparse) constraint matrix
val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
row = np.array([0, 0, 0, 1, 1])
col = np.array([0, 1, 2, 0, 1])

A = sp.csr_matrix((val, (row, col)), shape=(2, 3))

# Build rhs vector
rhs = np.array([4.0, -1.0])

# Add constraints
m.addConstr(A @ x <= rhs, name="c")

#save model information before optimize
m.write('matrix.lp') #for model browsing
m.write('matrix.mps') #for model reuse
m.write('matrix.prm') #save parameters

# Optimize model
m.optimize()

#save Solution information in Json File
m.write('matrix_sol.json')




print(x.X)
print('Obj: %g' % m.ObjVal)



Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x04118249
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.000000000e+00
[4. 0. 0.]
Obj: 4


## Load Gurobi Model File to Solve and Check Model Components

## The lp file also could be directly read to see constraints

In [48]:
model = read('matrix.mps')
model.optimize()

Read MPS format model from file matrix.mps
Reading time = 0.00 seconds
matrix1: 2 rows, 3 columns, 5 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x04118249
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.000000000e+00


In [20]:
print(model.display())

Maximize
  <gurobi.LinExpr: x[0] + x[1] + 2.0 x[2]>
Subject To
  c[0]: <gurobi.LinExpr: x[0] + 2.0 x[1] + 3.0 x[2]> <= 4
  c[1]: <gurobi.LinExpr: -1.0 x[0] + -1.0 x[1]> <= -1
None


In [15]:
obj = model.getObjective()
print(obj.getValue())
print(obj)

4.0
<gurobi.LinExpr: x[0] + x[1] + 2.0 x[2]>


In [16]:
model.getVars()

[<gurobi.Var x[0] (value 4.0)>,
 <gurobi.Var x[1] (value 0.0)>,
 <gurobi.Var x[2] (value 0.0)>]

In [7]:
for v in model.getVars():
    print(v.Varname,v.X)

x[0] 4.0
x[1] 0.0
x[2] 0.0


In [17]:
#name c[0], name c[1]
model.getConstrs()

[<gurobi.Constr c[0]>, <gurobi.Constr c[1]>]

In [18]:
#only print nonzero solutions
model.printAttr('x')


    Variable            x 
-------------------------
        x[0]            4 


In [41]:
#just linear constraints
#other constraints similar function
for cnstr in model.getConstrs():
    print(cnstr.ConstrName)
    for var in model.getVars():
        print(var.Varname)
        print(model.getCoeff(cnstr, var))
    print('----')

c[0]
x[0]
1.0
x[1]
2.0
x[2]
3.0
----
c[1]
x[0]
-1.0
x[1]
-1.0
x[2]
0.0
----


## Load Json File to See Sol Information without Solve

In [46]:
f = open('matrix_sol.json')
sol = json.load(f)
print(sol)
f.close()

{'SolutionInfo': {'Status': 2, 'Runtime': 0.00764012336730957, 'Work': 2.5094666666666673e-06, 'ObjVal': 4, 'BoundVio': 0, 'ConstrVio': 0, 'IterCount': 0, 'BarIterCount': 0}, 'Vars': [{'VarName': 'x[0]', 'X': 4}]}


## Gurobi Log File
Directly read it from output path

## Gurobi Prm File: save parameters